# Using Soundscape Simulation to Evaluate Compositions for a Public Space Sound Installation

## Valérian Fraisse
 
The present code was used to generate reaper projects corresponding to randomized baseline audio reported in the journal article 'Using Soundscape Simulation to Evaluate Compositions for a Public Space Sound Installation' by Fraisse V., Schütz N., Wanderley M., Guastavino C., and Misdariis N., under publication to the Journal of the Acoustical Society of America. 

In [ ]:
import random
from reathon.nodes import *
import os
import contextlib
import wave
from itertools import groupby

data_dir = os.path.join(os.getcwd(), 'Excerpts')

In [2]:
sources = []
lengths = []
randlists = []
inlist = []

# Initialize number of excerpts
ne = 0
# Number of participants
np = 20
# Crossfade duration in seconds
xfade = 3



# Generating list of excerpts
for x in os.listdir(data_dir):
    if x.endswith('.wav'):
        ne += 1
        sources.append(Source(file = x))
        with contextlib.closing(wave.open(os.path.join(data_dir,x), 'r')) as f:
            lengths.append(f.getnframes()/float(f.getframerate()))
            
for i in range(ne):
    inlist.append(i)
    inlist.append(i)
            
for i in range(np):
    randwithdup=random.sample(inlist, k = ne*2)
    randlists.append([x for x,_ in groupby(randwithdup)])

In [3]:
p = 0
pos_list = []

# Generating .rpp files
for randlist in randlists:
    track = Track()
    pos = 0.0
    # Track parameters
    track.props = [
        ["NCHAN", 32],
        ["REC", "0 2112 1 0 0 0 0"],
        ["MAINSEND", "0 0"],
        ["HWOUT", "64 0 1 0 0 0 16384 -1:U -1"],
        ["VU", 2],
        ["TRACKHEIGHT", "450 0 0 0 0 0"],
         
    ]
    
    for i in randlist:
        pos_list.append(round(pos, 2))
        # Audio item parameters
        item = Item(sources[i])
        item.props = [
            ["LENGTH", lengths[i]],
            ["POSITION", pos],
            # Fade in/out settings
            # field 1, int, fade in curve type (?)
            # field 2, float, fade in time (in seconds) (?)
            # field 3, float, autocrossfade length
            # field 4, float, ? (bool)
            # field 5, float, autocrossfade is on
            # field 6, float, fade shape
            # field 7, float, ? (bool?)
            ["FADEIN", "1 0 " + str(xfade) + " 1 1 0 0"],
            ["FADEOUT", "1 0 " + str(xfade) + " 1 1 0 0"]
        ]
        track.add(item)
        pos += lengths[i] - xfade

        baseline_project = Project(track)
        
    baseline_project.write("baseline_exp1_P" + str(p+2) + ".rpp")
    p += 1